In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

import torchvision.transforms as tr 

from dataset import make_dataset, collate_fn

In [3]:
import sys
import os

sys.path.append(os.path.abspath('..'))

from data.dataset import make_dataset, collate_fn
from torch.utils.data import DataLoader

# 설정 변수
batch_size = 4
size = 512

image_dir_train = 'train/images/'
bbox_dir_train = 'train/bbox/'

image_dir_val = 'validation/images/'
bbox_dir_val = 'validation/bbox/'

image_dir_test = 'test/images/'
bbox_dir_test = 'test/bbox/'


# 데이터셋 생성
train_set = make_dataset(
    image_dir=image_dir_train,
    bbox_dir=bbox_dir_train
)

val_set = make_dataset(
    image_dir=image_dir_val,
    bbox_dir=bbox_dir_val
)

test_set = make_dataset(
    image_dir=image_dir_test,
    bbox_dir=bbox_dir_test
)

# DataLoader 생성
train_loader = DataLoader(
    train_set, 
    batch_size=batch_size,
    shuffle=True,
    pin_memory=True,
    collate_fn=collate_fn,
    num_workers=0
)

val_loader = DataLoader(
    val_set, 
    batch_size=batch_size,
    shuffle=False,
    pin_memory=True,
    collate_fn=collate_fn,
    num_workers=0
)

test_loader = DataLoader(
    test_set, 
    batch_size=batch_size,
    shuffle=True,
    collate_fn=collate_fn
)

# 데이터 로드 및 출력
images, labels, bbox = next(iter(val_loader))

print(images.shape)
print(labels)
print(bbox)

torch.Size([4, 3, 512, 512])
(tensor([4]), tensor([4]), tensor([4]), tensor([4]))
(tensor([[ 200.9026,    0.0000, 1158.6770,  960.9201]]), tensor([[ 266.9031,  348.4540, 1536.1183,  878.4917]]), tensor([[ 897.0312,    0.0000, 1280.0000,  720.0000]]), tensor([[ 348.3173,  163.5216, 1481.7118,  965.0387]]))


# Model

In [3]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.6/779.6 kB 13.9 MB/s eta 0:00:00 MB/s eta 0:00:01


In [5]:
from ultralytics import YOLO

In [3]:
import wandb
wandb.login(relogin=True)

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········


wandb: Appending key for api.wandb.ai to your netrc file: /home/team1/.netrc


True

In [13]:
!nvidia-smi

Fri Jun  7 20:34:04 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.29.06              Driver Version: 545.29.06    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA TITAN Xp                Off | 00000000:19:00.0 Off |                  N/A |
| 35%   59C    P2              83W / 250W |   2878MiB / 12288MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [6]:
model = YOLO('DL_Programming/yolov8n.pt')

In [6]:
print(type(model.names), len(model.names))
print(model.names)

<class 'dict'> 80
{0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted plant', 59: 'bed', 60: 'dining table', 61: 'toilet', 62: 'tv', 63: 'laptop', 64: 'mouse', 65: 'remote', 66: 'keyboard', 67: 'cell p

In [ ]:
model.train(data='data.yaml', epochs=50, patience=10, batch=128, imgsz=512, device=0)

Ultralytics YOLOv8.2.28 🚀 Python-3.10.14 torch-2.2.1+cu121 CUDA:0 (NVIDIA TITAN Xp, 12188MiB)
engine/trainer: task=detect, mode=train, model=DL_Programming/yolov8n.pt, data=data.yaml, epochs=50, time=None, patience=10, batch=128, imgsz=512, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train9, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, sh

wandb: Currently logged in as: jitoo6611 (interactive_ai). Use `wandb login --relogin` to force relogin


Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
AMP: checks passed ✅


train: Scanning /mnt/ddrive/team1/DL/DL_Programming/data/train/labels.cache... 8421 images, 0 backgrounds, 32 corrupt: 100%|██████████| 8421/8421 [00:00<?, ?it/s]

train: WARNING ⚠️ /mnt/ddrive/team1/DL/DL_Programming/data/train/images/bundle of ropes_009_00046.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0048      1.0029]
train: WARNING ⚠️ /mnt/ddrive/team1/DL/DL_Programming/data/train/images/bundle of ropes_010_00326.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0011      1.0029]
train: WARNING ⚠️ /mnt/ddrive/team1/DL/DL_Programming/data/train/images/bundle of ropes_010_00359.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0021]
train: WARNING ⚠️ /mnt/ddrive/team1/DL/DL_Programming/data/train/images/bundle of ropes_042_01171.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0011]
train: WARNING ⚠️ /mnt/ddrive/team1/DL/DL_Programming/data/train/images/bundle of ropes_042_07627.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0011]
train: WARNING ⚠️ /mn

...

                   all       2402       2402      0.771      0.653      0.675      0.523

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      18/50      11.8G     0.7981     0.9156      1.165        159        512: 100%|██████████| 66/66 [02:11<00:00,  1.99s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:31<00:00,  3.12s/it]

                   all       2402       2402       0.77      0.648      0.675      0.524



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      19/50      11.8G     0.7955     0.9281      1.168        182        512: 100%|██████████| 66/66 [01:43<00:00,  1.56s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  10%|█         | 1/10 [00:01<00:10,  1.16s/it]

In [ ]:
print(type(model.names), len(model.names))
print(model.names)

In [ ]:
results = model.predict(source='data\test\images', save=True)

In [27]:
import os
import xml.etree.ElementTree as ET
from PIL import Image
from typing import Type
import torch
import torchvision.transforms as tr
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, image_dir, bbox_dir, transform):
        self.image_dir = image_dir
        self.bbox_dir = bbox_dir
        self.data_images = [file for file in os.listdir(image_dir) if file.endswith('.jpg')]
        self.transform = transform

        # 레이블 매핑 딕셔너리
        self.label_map = {
            'tire': 0,
            'spring fish trap': 1,
            'circular fish trap': 1,
            'rectangular fish trap': 1,
            'eel fish trap': 1,
            'fish net': 2,
            'wood': 3,
            'rope': 4,
            'bundle of ropes': 4
        }

    def parse_bbox_xml(self, xml_file, image_width, image_height):
        tree = ET.parse(xml_file)
        root = tree.getroot()

        boxes = []
        labels = []
        for obj in root.findall('object'):
            name = obj.find('name').text
            label = self.label_map.get(name, -1)  # 레이블 매핑, 없으면 -1
            
            if label == -1:
                continue

            bbox = obj.find('bndbox')
            xmin = float(bbox.find('xmin').text) / image_width
            ymin = float(bbox.find('ymin').text) / image_height
            xmax = float(bbox.find('xmax').text) / image_width
            ymax = float(bbox.find('ymax').text) / image_height

            # 바운딩 박스의 중심점과 너비, 높이 계산
            x_center = (xmin + xmax) / 2
            y_center = (ymin + ymax) / 2
            width = xmax - xmin
            height = ymax - ymin

            boxes.append([x_center, y_center, width, height])
            labels.append(label)

        # 객체가 1개만 있는 경우만 반환
        if len(boxes) == 1 and len(labels) == 1:
            return boxes, labels
        else:
            return None, None

    def __getitem__(self, idx):
        img_file = self.data_images[idx]
        img_path = os.path.join(self.image_dir, img_file)
        image = Image.open(img_path).convert("RGB")
        image_width, image_height = image.size

        # xml 파일 경로 찾기
        xml_file = os.path.join(self.bbox_dir, img_file.replace('.jpg', '.xml'))

        boxes, labels = self.parse_bbox_xml(xml_file, image_width, image_height)
        
        # boxes 또는 labels가 None이면 다음 데이터로 넘어감
        if boxes is None or labels is None:
            return self.__getitem__((idx + 1) % len(self.data_images))

        boxes = torch.tensor(boxes, dtype=torch.float32)
        labels = torch.tensor(labels, dtype=torch.int64)

        image = self.transform(image)

        return image, labels, boxes

    def __len__(self):
        return len(self.data_images)

def make_dataset(image_dir: str,
                 bbox_dir: str,
                 transform=tr.Compose([tr.Resize((512, 512)), 
                                       tr.ToTensor(),                                       
                                       tr.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                                       ])
                 ) -> Type[torch.utils.data.Dataset]:
    """
    Make pytorch Dataset for given task.
    Read the image using the PIL library and return it as an np.array.

    Args:
        image_dir (str): dataset directory
        bbox_dir (str) : dataset directory
        transform (torchvision.transforms) pytorch image transforms  

    Returns:
        torch.Dataset: pytorch Dataset
    """
        
    dataset = CustomDataset(image_dir=image_dir,
                            bbox_dir=bbox_dir,
                            transform=transform)
            
    return dataset

def collate_fn(batch):
    images, labels, boxes = zip(*batch)
    images = torch.stack(images, 0)
    return images, labels, boxes

# 저장할 경로
image_dir_train = 'train/images/'
bbox_dir_train = 'train/bbox/'

image_dir_val = 'validation/images/'
bbox_dir_val = 'validation/bbox/'

image_dir_test = 'test/images/'
bbox_dir_test = 'test/bbox/'

# 라벨 파일 생성 및 저장
def save_label_file(image_file, labels, boxes, output_dir):
    label_file = image_file.replace('.jpg', '.txt')
    with open(os.path.join(output_dir, label_file), 'w') as f:
        for label, box in zip(labels, boxes):
            x_center, y_center, width, height = box
            line = f"{label} {x_center} {y_center} {width} {height}\n"
            f.write(line)

def save_label_files(dataset, image_dir, bbox_dir, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    for idx in range(len(dataset)):
        image, labels, boxes = dataset[idx]
        image_file = dataset.data_images[idx]
        save_label_file(image_file, labels, boxes, output_dir)

# train set
train_dataset = make_dataset(image_dir_train, bbox_dir_train)
save_label_files(train_dataset, image_dir_train, bbox_dir_train, 'train/labels')

# validation set
val_dataset = make_dataset(image_dir_val, bbox_dir_val)
save_label_files(val_dataset, image_dir_val, bbox_dir_val, 'validation/labels')

# test set
# test_dataset = make_dataset(image_dir_test, bbox_dir_test)
# save_label_files(test_dataset, image_dir_test, bbox_dir_test, 'test/labels')

In [26]:
import cv2
import numpy as np

def read_yolo_label(label_file):
    with open(label_file, 'r') as f:
        lines = f.readlines()
    
    labels = []
    for line in lines:
        parts = line.strip().split()
        class_id, x_center, y_center, width, height = map(float, parts)
        labels.append((int(class_id), x_center, y_center, width, height))
    
    return labels

def draw_bounding_boxes(image, labels):
    h, w, _ = image.shape
    for class_id, x_center, y_center, width, height in labels:
        x_min = int((x_center - width/2) * w)
        y_min = int((y_center - height/2) * h)
        x_max = int((x_center + width/2) * w)
        y_max = int((y_center + height/2) * h)
        
        cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
        cv2.putText(image, f'Class: {class_id}', (x_min, y_min - 10), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    
    return image

# 이미지 및 레이블 파일 경로
image_path = 'test/images/wood_048_04863.jpg'
label_file = 'test/labels/wood_048_04863.txt'

# 이미지 읽기
image = cv2.imread(image_path)
if image is None:
    print(f"이미지를 불러올 수 없습니다: {image_path}")
    exit()

# 레이블 읽기
labels = read_yolo_label(label_file)

# Bounding box 그리기
image_with_boxes = draw_bounding_boxes(image.copy(), labels)

# 결과 이미지 표시
cv2.imshow('Image with Bounding Boxes', image_with_boxes)
cv2.waitKey(0)
cv2.destroyAllWindows()